In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [ ]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
df = pd.read_csv('/content/GE_train_data.csv', usecols = ['datetime', 'energy'], encoding = 'latin1')
df.head()

In [ ]:
df.dropna(subset = ['datetime'])

In [ ]:

# Convert 'Order_Date' to datetime
df['datetime'] = pd.to_datetime(df['datetime'], format='mixed')

In [ ]:
df['energy'] = df['energy'].interpolate(method='linear')

In [ ]:
# Calculate the frequency dynamically
freq = (df['datetime'].iloc[-1] - df['datetime'].iloc[-2]).total_seconds()

# Generate the next 10,000 entries
start_date = df['datetime'].iloc[-1]
new_dates = pd.date_range(start=start_date + pd.Timedelta(seconds=freq),
                          periods=17544,
                          freq=f"{int(freq)}S")  # Frequency in seconds

# Append the new entries to the DataFrame
future_df = pd.DataFrame({'datetime': new_dates})
future_df

In [ ]:
# Define a function for date-related feature engineering
def extract_date_features(data):
    # Extract basic date components
    data['year'] = data['datetime'].dt.year
    data['is_leap_year'] = data['datetime'].dt.is_leap_year.astype(int)
    data['week_of_year'] = data['datetime'].dt.isocalendar().week
    data['day_of_year'] = data['datetime'].dt.dayofyear

    data['quarter'] = data['datetime'].dt.quarter

    data['month'] = data['datetime'].dt.month
    data['day_of_month'] = data['datetime'].dt.day
    data['week_of_month'] = data['datetime'].dt.isocalendar().week % 4 + 1

    data['day_of_week'] = data['datetime'].dt.dayofweek
    data['is_weekend'] = data['datetime'].dt.dayofweek.isin([5, 6]).astype(int)

    data['hour'] = data['datetime'].dt.hour
    # data['minute'] = data['datetime'].dt.minute

    # Add sine and cosine transformations for cyclical features
    # Month
    data['month_sin'] = np.sin(2 * np.pi * data['datetime'].dt.month / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['datetime'].dt.month / 12)

    # Day of the week
    data['day_of_week_sin'] = np.sin(2 * np.pi * data['datetime'].dt.dayofweek / 7)
    data['day_of_week_cos'] = np.cos(2 * np.pi * data['datetime'].dt.dayofweek / 7)

    # Hour
    data['hour_sin'] = np.sin(2 * np.pi * data['datetime'].dt.hour / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['datetime'].dt.hour / 24)

    return data

In [ ]:
# Extract features and target
df = extract_date_features(df)  # Ensure features are added for training
df

In [ ]:

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['energy'].quantile(0.25)
Q3 = df['energy'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Create a binary column for IQR outliers
df['is_iqr_outlier'] = ((df['energy'] < lower_bound) | (df['energy'] > upper_bound)).astype(int)

In [ ]:
from scipy.stats import zscore

# Calculate Z-Score for the energy column
df['z_score'] = zscore(df['energy'])

# Define Z-Score threshold for 99% confidence
z_threshold = 2.576

# Create a binary column for Z-Score outliers
df['is_z_outlier'] = (abs(df['z_score']) > z_threshold).astype(int)

# Drop the temporary 'z_score' column if not needed
df.drop(columns=['z_score'], inplace=True)

In [ ]:
# Compute max and min of the 'energy' column (excluding outliers)
max_value = df.loc[
    (df['is_iqr_outlier'] == 0) & (df['is_z_outlier'] == 0), 'energy'
].max()

min_value = df.loc[
    (df['is_iqr_outlier'] == 0) & (df['is_z_outlier'] == 0), 'energy'
].min()

# Replace outliers in 'energy' column
df['energy'] = np.where(
    (df['is_iqr_outlier'] == 1) & (df['is_z_outlier'] == 1),
    np.where(df['energy'] > max_value, max_value, min_value),
    df['energy']
)

df

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
target_col = 'energy'

In [ ]:
# Split data into train and test sets by time order
train_test_fraction = 0.88
train_size = int(len(df) * train_test_fraction)

# Splitting features and target
X_train, X_test = df.iloc[:train_size].drop(columns=target_col), df.iloc[train_size:].drop(columns=target_col)
Y_train, Y_test = df.iloc[:train_size][target_col], df.iloc[train_size:][target_col]

In [ ]:
# Fit and transform specific columns
X_train[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']] = scaler.fit_transform(
    X_train[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']]
)


In [ ]:
# Transform specific columns
X_test[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']] = scaler.transform(
    X_test[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']]
)


In [ ]:
# Compute correlations, take absolute values, and sort by 'energy' in decreasing order
corr_abs_sorted = pd.concat([X_train.drop(['datetime', 'is_z_outlier', 'is_iqr_outlier'], axis=1), Y_train], axis=1).corr()['energy'].abs().sort_values(ascending=False)
corr_abs_sorted

In [ ]:
# Select the top 10 features (excluding 'energy' itself)
feature_cols = corr_abs_sorted.drop(['energy']).index.tolist()  # Exclude the first entry (self-correlation)

feature_cols

In [ ]:
x_train = X_train[feature_cols].values.astype(np.float32)

In [ ]:
x_test = X_test[feature_cols].values.astype(np.float32)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:

y_train = Y_train.values
y_train

In [ ]:
y_test = Y_test.values
y_test

In [ ]:

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.float32)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [ ]:
train_dataset = CustomDataset(x_train, y_train)

In [ ]:
test_dataset = CustomDataset(x_test, y_test)

In [ ]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):

    return self.model(x)

In [ ]:
# Set device to CPU or GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set random seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set random seed for other libraries
np.random.seed(42)


In [ ]:
best_modal_state = None  # Global variable to track the best modal state
best_mape = float('inf')  # Initialize best MAPE with positive infinity (high value)


# Objective function
def objective(trial):
    global best_modal_state, best_mape

    # Hyperparameter suggestions
    # Model Hyperparameters
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 7)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 64, 256, step=8)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

    # Training Hyperparametrs
    epochs = trial.suggest_int("epochs", 20, 100, step=10)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    # Model initialization
    input_dim = train_dataset.features.shape[1]  # Dynamically set input dimensions
    output_dim = 1  # Regression task
    model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate).to(device)

    # Loss and optimizer
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)



    import torch
    import torch.nn.functional as F

    # Shifting model to device
    model = model.to(device)

    # Initialize a list to store epoch-wise MAPE
    epoch_mape_list = []

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_mape_train = 0  # Initialize total MAPE for the epoch
        batch_count_train = 0  # Initialize batch counter

        for batch_features, batch_labels in train_loader:
            # Move data to device and ensure float32 dtype
            batch_features = batch_features.to(device, dtype=torch.float32)
            batch_labels = batch_labels.to(device, dtype=torch.float32)

            # Forward pass
            outputs = model(batch_features)

            # Calculate loss
            loss = F.l1_loss(outputs, batch_labels.unsqueeze(1))

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate training MAPE
            mape = torch.mean(torch.abs((outputs - batch_labels.unsqueeze(1)) / (batch_labels.unsqueeze(1) + 1e-8)) * 100)
            total_mape_train += mape.item()  # Accumulate MAPE for the epoch
            batch_count_train += 1  # Increment batch counter

        # Average MAPE for the epoch
        avg_mape_train_epoch = total_mape_train / batch_count_train
        epoch_mape_list.append(avg_mape_train_epoch)  # Store the epoch MAPE

    # Calculate the average MAPE across all epochs
    avg_mape_all_epochs = sum(epoch_mape_list) / len(epoch_mape_list)
    print(f"\nAverage Train MAPE Across All Epochs: {avg_mape_all_epochs:.2f}%")



    # Evaluation loop
    model.eval()
    total_mape = 0
    batch_count = 0

    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            # Move data to device
            batch_features = batch_features.to(device, dtype=torch.float32)
            batch_labels = batch_labels.to(device, dtype=torch.float32)

            # Forward pass
            outputs = model(batch_features)

            # Calculate batch-wise MAPE
            batch_mape = torch.mean(torch.abs((outputs - batch_labels.unsqueeze(1)) / batch_labels.unsqueeze(1)) * 100)
            total_mape += batch_mape.item()
            batch_count += 1

    # Average MAPE for the test set
    avg_mape = total_mape / batch_count

    if avg_mape < best_mape:
      best_mape = avg_mape
      best_modal_state = model.state_dict() # Save the best modal's weights

    trial.set_user_attr("best_mape", best_mape)
    print(f"Average Test MAPE: {avg_mape:.2f}%")
    return avg_mape


In [ ]:
!pip install optuna

In [ ]:
import optuna

study = optuna.create_study(direction='minimize')

In [ ]:
study.optimize(objective, n_trials=10)

In [ ]:
# Initialize the best model with best hyperparameter
best_trial = study.best_trial
best_model = MyNN(
    input_dim = train_dataset.features.shape[1],  # Dynamically set input dimension,
    output_dim = 1,
    num_hidden_layers = best_trial.params["num_hidden_layers"],
    neurons_per_layer = best_trial.params["neurons_per_layer"],
    dropout_rate = best_trial.params["dropout_rate"]
).to(device)

# Load the saved best modal's weights from the best trial
best_model.load_state_dict(best_modal_state)

In [ ]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial hyperparameters:", best_trial.params)
print("Best trial value (Accuracy / Error) :", best_trial.value)

In [ ]:
def evaluate_model(model, dataset):
    best_model.eval()  # Set the model to evaluation mode
    mape_list = []  # To store MAPE for each input

    with torch.no_grad():
        for features, labels in DataLoader(dataset, batch_size=1, shuffle=False):
            # Move features and labels to the device
            features = features.to(device, dtype=torch.float32)
            labels = labels.to(device, dtype=torch.float32)

            # Forward pass
            outputs = model(features)

            # Compute MAPE with clamped labels to avoid division issues
            mape = torch.abs((outputs - labels) / torch.clamp(labels, min=1e-2)) * 100
            mape_list.append(mape.item())

    # Calculate average MAPE
    avg_mape = sum(mape_list) / len(mape_list)
    return mape_list, avg_mape

In [ ]:
# Evaluate on training data
train_mape_list, train_avg_mape = evaluate_model(best_model, train_dataset)

In [ ]:
# Evaluate on test data
test_mape_list, test_avg_mape = evaluate_model(best_model, test_dataset)

In [ ]:
# Print MAPE for each sample and average MAPE
print("MAPE for each sample in Training Data:")
for i, mape in enumerate(train_mape_list):
    f"Sample {i + 1}: {mape:.4f}%"
print(f"\nAverage MAPE for Training Data: {train_avg_mape:.4f}%")


In [ ]:

print("\nMAPE for each sample in Test Data:")
for i, mape in enumerate(test_mape_list):
    f"Sample {i + 1}: {mape:.4f}%"
print(f"\nAverage MAPE for Test Data: {test_avg_mape:.4f}%")

In [ ]:
import pandas as pd

# Function to apply the model and create a DataFrame with actual and forecasted values
def evaluate_and_create_df(model, dataset, dataset_name):
    best_model.eval()
    results = {"Actual": [], "Forecasted": []}  # Dictionary to store results

    with torch.no_grad():
        for features, labels in DataLoader(dataset, batch_size=1, shuffle=False):
            # Move features and labels to the device
            features = features.to(device, dtype=torch.float32)
            labels = labels.to(device, dtype=torch.float32)

            # Forward pass
            outputs = model(features)

            # Append actual and forecasted values to the dictionary
            results["Actual"].append(labels.item())         # Convert label to scalar
            results["Forecasted"].append(outputs.item())    # Convert output to scalar

    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    results_df["Dataset"] = dataset_name  # Add a column for dataset identification

    return results_df

# Apply the best model to training and testing datasets
train_results_df = evaluate_and_create_df(best_model, train_dataset, "Train")
test_results_df = evaluate_and_create_df(best_model, test_dataset, "Test")


In [ ]:
print(train_dataset.features[0])
print(train_dataset.labels[0])

In [ ]:
best_model(train_dataset[0][0].unsqueeze(0).to(device, dtype=torch.float32))

In [ ]:

# Combine training and testing results into a single DataFrame
results_df = pd.concat([train_results_df, test_results_df], ignore_index=True)
final_results_df = pd.concat([df['datetime'], results_df], axis=1)

# Save or print the final DataFrame
print(final_results_df)

In [ ]:
future_df

In [ ]:
full_dates_df = pd.concat([df[['datetime']], future_df[['datetime']]], axis=0).reset_index(drop=True)

full_dates_df

In [ ]:
full_feature_dates_df = extract_date_features(full_dates_df)
full_feature_dates_df

In [ ]:
# Fit and transform specific columns
full_feature_dates_df[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']] = scaler.transform(
    full_feature_dates_df[['year', 'week_of_year', 'quarter', 'day_of_year', 'month', 'day_of_month', 'week_of_month', 'day_of_week', 'hour']])
full_feature_dates_df

In [ ]:
full_date_features_df = full_feature_dates_df[feature_cols]

full_date_features_df

In [ ]:
# Ensure the model is in evaluation mode
best_model.eval()

# Convert features to PyTorch tensor and move to the device
features_tensor = torch.tensor(full_date_features_df.values.astype(np.float32), dtype=torch.float32).to(device)

# Use the model to predict
with torch.no_grad():  # No gradients are needed for inference
    y_pred = best_model(features_tensor)

In [ ]:
print(type(y_pred))

In [ ]:
# Move predictions to CPU if they are on a GPU, then convert to NumPy or list
y_pred = y_pred.cpu().numpy()  # For a NumPy array

y_pred_list = y_pred.tolist()  # For a Python list

In [ ]:
y_pred

In [ ]:

y_pred_df = pd.DataFrame(y_pred, columns=['energy'])

In [ ]:
y_pred_df

In [ ]:
full_dates_df['datetime']

In [ ]:
pred_df = pd.concat([full_dates_df['datetime'], y_pred_df], axis=1)
pred_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set 'datetime' as the index for both DataFrames
df.set_index('datetime', inplace=True)
pred_df.set_index('datetime', inplace=True)

In [ ]:

# Resample the data by month, summing the energy values
df_monthly = df['energy' ].resample('ME').sum()
pred_df_monthly = pred_df.resample('ME').sum()

In [ ]:
df_monthly

In [ ]:
pred_df_monthly

In [ ]:
merged = pd.concat([df_monthly, pred_df_monthly], axis=1, keys=['Actual', 'Predicted'])
merged

In [ ]:
import matplotlib.pyplot as plt

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(merged.index, merged[('Actual', 'energy')], label='Actual Energy', color='blue', marker='o')
plt.plot(merged.index, merged[('Predicted', 'energy')], label='Predicted Energy', color='orange', linestyle='--', marker='x')

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Energy')
plt.title('Energy Consumption Forecasting: Actual vs Predicted by Month')

# Show legend
plt.legend(loc='lower right')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Set the background color to light green
plt.gcf().set_facecolor('lightgreen')

# Add text for train-test split and data ranges
info_text = (
    "MAPE = 6%\n"
    "Train Data: 2008-2016\n"
    "Test Data: 2016-2018\n"
    "Predicted Data: 2018-2020\n"
    "Data: Hourly, Grouped by Month"
)
plt.text(0.02, 0.95, info_text, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgrey'))

# Display the plot
plt.tight_layout()
plt.show()